In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Mục đích của Notebook.

Notebook này sẽ thực hiện nhiệm vụ biến đổi dữ liệu đầu vào thành các đặc trưng khác nhau. Sau đó thực hiện tính feature importance để giảm số lượng feature phải lấy.

## Định nghĩa các số liệu đầu vào

In [ ]:
%%time

# Thư viện để trực quan hóa tiến độ.
from tqdm import tqdm_notebook as tqdm

# Tổng số dòng dữ liệu
total = 629145481 

# Khi sử dụng chunksize càng lớn, thì dữ liệu nạp càng nhanh.
# 2700000 là số lớn nhất chia hết cho cả 18 và 150 000. Mà 18 lại là ước của 4194 là số lần load. 
# Do đó, 2700000 được chọn làm chunksize.
chunksize = 2700000

# Tải dữ liệu sử dụng chunk, do nếu tải hết dữ liệu vào thì phần Editing sẽ báo lỗi.
chunks = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}, chunksize = chunksize)

c = 1

segments = total // chunksize
# for debug
# segments = 1
    
# Segment là số lượng đoạn được lấy từ input.
# Các chunk được tải sẽ được nối lại bằng làm append.

train = None

for i in tqdm(range(segments)):
    chunk = next(chunks)
    
    if train is None:
        train = chunk
    else:
        train = pd.concat([train, chunk])
    del chunk

In [ ]:
# Hiển thị đến độ chính xác 15 chữ số sau dấu phẩy.
pd.options.display.precision = 15

In [ ]:
# In ra 5 dòng đầu tiên của acoustic data
train[:5]

# Feature engineering

## Chuẩn bị các thư viện sẽ sử dụng.

In [ ]:
%%time
# Load các thư viện liên quan đến biến đổi tín hiệu và thống kê sẽ được sủ dụng.
from scipy import stats
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve

# Load thư viện về hồi quy tuyến tính đơn giản để tìm xu hướng của dữ liệu.
from sklearn.linear_model import LinearRegression

import numpy as np

In [ ]:
%%time

# Lọc bỏ các warning để tập trung hơn vào tiến độ xử lý dữ liệu.
import warnings
warnings.filterwarnings('ignore')

def classic_sta_lta(x, length_sta, length_lta):
    """
    Input: x là một mảng số.
    STA: Short Term Avg.
    LTA: Long Term Avg.
    
    - Term Average: được tính bằng thương độ khác nhau của các phần tử cách nhau một khoảng cố định trên khoảng cố định đó.
    - Short Term Avg: Term Average với khoảng nhỏ.
    - Long Term Avg: Term Average với khoảng lớn.
    """
    # Tổng cộng dồn các phần tử bình phương.
    sta = np.cumsum(x ** 2)

    # Chuyển đổi sang kiểu float.
    sta = np.require(sta, dtype=np.float)

    # Tương tự với mảng lta
    lta = sta.copy()

    # Tính STA là LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Tiếp theo ta tiến hành chia sta cho lta. Các giá trị nan sẽ được biến thành 0.
    sta[:length_lta - 1] = 0

    # Định nghĩa dtiny là số nhỏ nhất mà máy tính có thể chứa.
    dtiny = np.finfo(0.0).tiny
    
    # Với mỗi lta nhỏ hơn dtiny, sẽ đặt bằng dtiny.
    idx = lta < dtiny
    lta[idx] = dtiny
    
    return sta / lta
    

def get_feature(ad):
    """
    Đầu vào: ad (Acoustic Data): pd.Series: chứa 150 000 dòng dữ liệu.
    Đầu ra:
    - Các thông số thống kê của ad. Được tính cho cả ad và trị tuyệt đối của ad.
    """
    features = dict()
    ## Trung bình, đồ lệch chuẩn, max, min và khoảng cách giữa max và min của ad.
    features['ave'] = ad.mean()
    features['std'] = ad.std() 
    features['max'] = ad.max()
    features['min'] = ad.min() 
    features['min_max_diff'] = features['max'] - features['min']

    features[ 'abs_ave'] = np.abs(ad).mean()
    features['abs_std'] = np.abs(ad).std()
    features['abs_max'] = np.abs(ad).max() 
    features['abs_min'] = np.abs(ad).min()
    features['abs_min_max_diff'] = features['abs_max'] - features['abs_min']

    ## Số lượng thời điểm ad có giá trị > 400.
    features['count_abnormal'] = len(ad[np.abs(ad) > 400])
    ## Tổng các ad.
    features['sum'] = ad.sum()

    # Tìm xu hướng của ad sử dụng hồi quy tuyến tính.
    # Thuật toán hồi quy tuyến tính sẽ được nói rõ trong notebook về thực nghiệm.
    model = LinearRegression().fit(np.arange(len(ad)).reshape(-1, 1), ad)

    features['coef_'] = model.coef_[0]
    
    model = LinearRegression().fit(np.arange(len(ad)).reshape(-1, 1), np.abs(ad))

    features['abs_coef_'] = model.coef_[0]
    
    # Chia ad thành các phần nhỏ hơn và tính mean, std, max, min và khoảng cách giữa max và min.
    div = 5
    for i in range(div):
        features['ave_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].mean()
        features['std_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].std()
        features['max_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].max()
        features['min_div_' + str(i)] = ad[rows // div * i: rows // div * (i + 1)].min()
        features['min_max_diff_div_' + str(i)] = features['max_div_' + str(i)] - features['min_div_' + str(i)]

        features['abs_ave_div_' + str(i)] = np.abs(ad[rows // div * i: rows // div * (i + 1)]).mean()
        features['abs_std_div_' + str(i)] = np.abs(ad[rows // div * i: rows // div * (i + 1)]).std()
        features['abs_max_div_' + str(i)] = np.abs(ad[rows // div * i: rows // div * (i + 1)]).max() 
        features['abs_min_div_' + str(i)] = np.abs(ad[rows // div * i: rows // div * (i + 1)]).min()
        features['abs_min_max_diff_div_' + str(i)] = features['abs_max_div_' + str(i)] - features['abs_min_div_' + str(i)]  

    # trung bình điều hòa. x_i phải dương.
    # H = n / (1/x_1 + 1/x_2 + ... + 1/x_n)
    features['hmean'] = stats.hmean(np.abs(ad[ad != 0]))
    # Trung bình nhân. x_i phải dương.
    # G = power(x_1 * x_2 * ... * x_n, 1/n)
    features['gmean'] = stats.gmean(np.abs(ad[ad != 0]))
    
    # Quantile: lượng tử. Liên kết với một tỉ lệ k.
    # Là điểm mà các vị trí dưới nó có tổng tỉ lệ xuất hiện với xác suất k.
    
    # Percentile là quantile sử dụng các tỉ lệ được chia theo các số nguyên của 100%.
    percentages = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    for percentage in percentages:
        features['percentile_' + str(percentage)] = np.percentile(ad, percentage)
        
    for quantile in percentages:
        features['quantile_' + str(quantile)] = np.quantile(ad, quantile)
        
    for percentage in percentages:
        features['abs_percentile_' + str(percentage)] = np.percentile(np.abs(ad), percentage)
        
    for quantile in percentages:
        features['abs_quantile_' + str(quantile)] = np.quantile(np.abs(ad), quantile)
        
    # Median absolute deviation: Trung bình trị tuyệt đối của độ lệch.
    features['mad'] = np.mean(np.abs(ad - np.mean(ad)))
    # Độ lệch:
    features['skew'] = stats.skew(ad)
    # Độ nhọn
    features['kurtosis'] = stats.kurtosis(ad)
    # Trung vị,
    features['median'] = np.median(ad)
    
    features['abs_mad'] = np.mean(np.abs(np.abs(ad) - np.mean(np.abs(ad))))
    features['abs_skew'] = stats.skew(np.abs(ad))
    features['abs_kurtosis'] = stats.kurtosis(np.abs(ad))
    features['abs_median'] = np.median(np.abs(ad))
    
    features['hilbert_mean'] = np.abs(hilbert(ad)).mean()
    
    # Phép làm min sử dụng vector Hann.
    # Tính tích chập giữa ad và vector hann, sau đó chia cho tổng của vector hann và tính mean.
    windows = [10, 50, 100, 200, 500, 1000, 5000, 10000]
    for window in windows:
        features['hann_window_' + str(window)] = (convolve(ad, hann(window), mode='same') / sum(hann(window))).mean()
    
    # Tính các đặc tính theo tổng liên tiếp (rolling)
    for window in windows:
        # pd.Series.rolling nếu dùng hàm mặc định sẽ trả về tổng các phần tử liên tiếp trong một khoảng window.
        roll_mean = ad.rolling(window).mean().dropna()
        roll_std = ad.rolling(window).std().dropna()
        
        # Tính mean, min, max, std, percentile trên trên tập roll_mean và roll_std.
        
        features['rolling_max_ave_' + str(window)] = roll_mean.max()
        features['rolling_min_ave_' + str(window)] = roll_mean.min()
        
        features['rolling_max_std_' + str(window)] = roll_std.max()
        features['rolling_min_std_' + str(window)] = roll_std.min()
        
        features['abs_rolling_max_std_' + str(window)] = np.abs(roll_std).max()
        features['abs_rolling_min_std_' + str(window)] = np.abs(roll_std).min()
        
        features['abs_rolling_max_mean_' + str(window)] = np.abs(roll_mean).max()
        features['abs_rolling_min_mean_' + str(window)] = np.abs(roll_mean).min()
        
        for percentage in percentages:
            features['abs_rolling_mean_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_mean, percentage)
            features['abs_rolling_std_{}_percentage_{}'.format(window, percentage)] = np.percentile(roll_std, percentage)
            
    # tính các đặc tính theo hiệu liên tiếp
    dif = ad.diff().dropna().values
    
    features['dif_mean'] = dif.mean()
    
    for percentage in percentages:
        features['dif_percentage_{}'.format(percentage)] = np.percentile(dif, percentage)
        features['dif_percentage_{}'.format(percentage)] = np.percentile(dif, percentage)
        
    # Tính trunh bình mức độ thay đổi giữa các giá trị liên tiếp trong ad.
    change = (dif / ad.values[:-1])
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    
    features['change_mean'] = change.mean()
    # sử dụng short term average và long term average
    
    features['classic_sta_lta_mean_1'] = classic_sta_lta(ad, 50, 10000).mean()
    features['classic_sta_lta_mean_1'] = classic_sta_lta(ad, 200, 10000).mean()
    features['classic_sta_lta_mean_1'] = classic_sta_lta(ad, 500, 10000).mean()
    features['classic_sta_lta_mean_1'] = classic_sta_lta(ad, 1000, 10000).mean()
    
    # sử dụng exponential Moving Average
    # Là phương pháp làm mịn phổ biến của xử lý tín hiệu
    for window in windows:
        features['EMA_{}'.format(window)] = ad.ewm(span=window).mean(skipna=True).mean(skipna=True)
    
    # Sử dụng Interquartile range
    # Là khoảng cách giữa percentile .75 và percentile .25.
    features['iqr'] = np.subtract(*np.percentile(ad, [75, 25]))
    
    # xóa biến để giải phóng bộ nhớ
    del change
    del dif
    
    return features


rows = 150000
segments = len(train) // rows

X = []
y = []

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]

    ad = seg['acoustic_data']
    ttf = seg['time_to_failure'].values[-1]
    
    y.append(ttf)
    
    ## Tính với dữ liệu chính, dữ liệu thực và phức của fft.
    
    ft = np.fft.fft(ad)
    real_ft = pd.Series(np.real(ft))
    imag_ft = pd.Series(np.imag(ft))
    
    # Nhận một dict() các đặc trưng.
    normal_feature = get_feature(ad)
    real_feature = get_feature(real_ft)
    img_feature = get_feature(imag_ft)
    
    # Nối các dict() lại với nhau.
    for key, value in real_feature.items():
        normal_feature['real_' + key] = value
    for key, value in img_feature.items():
        normal_feature['imag_' + key] = value
    
    X.append(normal_feature)

In [ ]:
X = pd.DataFrame(X)
y = pd.Series(y)

### Chạy mô hình catboost cơ bản để tính feature importance

Tiền xử lý dữ liệu sử dụng StandardScaler.

StandardScaler sẽ đưa dữ liệu về mean 0 và std 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, train_size=0.7)

Định nghĩa một mô hình catboost cơ bản.

In [ ]:
%%time
import catboost as cb

train_data = cb.Pool(X_train, y_train)
valid_data = cb.Pool(X_valid, y_valid)

model = cb.CatBoostRegressor(
    loss_function='MAE', 
    logging_level="Silent",
    iterations=300,
    learning_rate=0.1,
    depth=8,
    l2_leaf_reg=0.2,
    random_seed=500
)
model.fit(
    train_data,             
    verbose=False
)

Tính và trực quan hóa feature importance.

Giữ lại 120 feature có độ quan trọng cao nhất.

In [ ]:
feature_importance = model.get_feature_importance(
    train_data
)

feature_importance = pd.DataFrame({
    "feature_name": X.columns,
    "score": feature_importance
})

feature_importance.sort_values(["score"], axis=0, ascending=False, inplace=True)

keep_feature = feature_importance[feature_importance['score'] > 0.00001]
keep_feature = keep_feature.iloc[:120]
keep_feature

In [ ]:
for feature_name in keep_feature['feature_name']:
    print(feature_name)
    
keep_feature['feature_name'].values

Các thuộc tính này sẽ được giữ lại để làm đặc trưng. Cho các notebook sau.